In [ ]:
#Здесь я читаю все xlsx и сохраняю их в csv, т.к. читаются они около 22 мин, чтобы потом быстрее их читать 
import pandas as pd
import os
import time

# Засекаем время начала работы
start_time = time.time()

# Путь к распакованной директории
folder_path = './Files_about_flights/YourBoardingPassDotAero'

# Создадим список для хранения всех DataFrame'ов из всех файлов и листов
all_data = []

# Проходим по каждому файлу в директории
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.xlsx'):
            file_path = os.path.join(root, file)

            # Читаем все листы в Excel-файле
            sheets = pd.read_excel(file_path, sheet_name=None, header=None)

            # Проходим по каждому листу
            for sheet_name, data in sheets.items():
                try:
                    # Извлекаем нужные данные из листа
                    name = data.iloc[2, 1]
                    sequence = data.iloc[0, 7]
                    flight_number = data.iloc[4, 0]
                    departure_city = data.iloc[4, 3]
                    arrival_city = data.iloc[4, 7]
                    departure_airport = data.iloc[6, 3]
                    arrival_airport = data.iloc[6, 7]
                    flight_time = data.iloc[8, 2]
                    flight_date = data.iloc[8, 0]
                    seat = data.iloc[10, 7]
                    additional_info = data.iloc[10, 0]
                    pnr = data.iloc[12, 1]

                    # Преобразуем номер билета в строку, чтобы избежать экспоненциальной записи
                    ticket_number = str(data.iloc[12, 4])

                    # Создаем DataFrame для текущего листа
                    df = pd.DataFrame({
                        'Full Name': [name],
                        'SEQUENCE': [sequence],
                        'Flight Number': [flight_number],
                        'DepartureCity': [departure_city],
                        'DepartureAirport': [departure_airport],
                        'ArrivalCity': [arrival_city],
                        'ArrivalAirport': [arrival_airport],
                        'FlightTime': [flight_time],
                        'Flight Date': [flight_date],
                        'Seat': [seat],
                        'PaxAdditionalInfo': [additional_info],
                        'BookingCode': [pnr],
                        'TicketNumber': [ticket_number]
                    })

                    # Добавляем DataFrame в общий список
                    all_data.append(df)

                except Exception as e:
                    print(f"Ошибка при обработке листа {sheet_name} в файле {file}: {e}")

# Объединяем все DataFrame'ы в один большой
final_df = pd.concat(all_data, ignore_index=True)

# Преобразуем колонку 'TicketNumber' в строку (на всякий случай)
final_df['TicketNumber'] = final_df['TicketNumber'].astype(str)

# Сохраняем результат в CSV без кавычек и экспоненциальной записи
final_df.to_csv('xlsx_data.csv', index=False)

# Вывод времени работы программы
end_time = time.time()
execution_time = end_time - start_time
print(f"Время работы программы: {execution_time:.2f} секунд")

# Печатаем результат
final_df.head()

In [ ]:
#Последняя рабочая версия #Последняя рабочая версия #Последняя рабочая версия
#Последняя рабочая версия #Последняя рабочая версия #Последняя рабочая версия

import pandas as pd
import time
import json
import xml.etree.ElementTree as ET

# Засекаем время начала работы
start_time = time.time()

# Пути к файлам
csv_file_path = "./Files_about_flights/BoardingData.csv"
json_file_path = "./Files_about_flights/FrequentFlyerForum-Profiles.json"
tab_file_path = "./Files_about_flights/Sirena-export-fixed.tab"
xml_file_path = "./Files_about_flights/PointzAggregator-AirlinesData.xml"
xlsx_data_path = "./xlsx_data.csv"
output_file_path = "./Files_about_flights/results.csv"

# Шаг 1: Чтение данных из CSV-файла
df_csv = pd.read_csv(csv_file_path, sep=';', dtype={'PassengerDocument': 'object'})

# Переименование колонок CSV для унификации с JSON
df_csv = df_csv.rename(columns={
    'PassengerFirstName': 'First Name',
    'PassengerSecondName': 'Second Name',
    'PassengerLastName': 'Last Name',
    'PassengerSex': 'Sex',
    'PassengerBirthDate': 'Birth Date',
    'PassengerDocument': 'Document',
    'FlightNumber': 'Flight Number',
    'CodeShare': 'Share Code',
    'FlightDate': 'Flight Date',
    'Destination': 'ArrivalCity'
})

# Шаг 2: Чтение данных из JSON-файла
with open(json_file_path, 'r') as f:
    data = json.load(f)

# Преобразование данных JSON в pandas DataFrame
profiles = data["Forum Profiles"]

parsed_data = []
for profile in profiles:
    first_name = profile["Real Name"]["First Name"]
    last_name = profile["Real Name"]["Last Name"]
    sex = profile["Sex"]
    document = profile["Travel Documents"][0]["Passports"]
    
    for flight in profile["Registered Flights"]:
        if profile.get("Loyality Programm"):
            loyality_program = profile["Loyality Programm"][0]
            loy_prgrm_status = loyality_program.get("Status", None)
            loy_prgrm_code = loyality_program.get("programm", None)
            loy_prgrm_number = loyality_program.get("Number", None)
        else:
            loy_prgrm_status = loy_prgrm_code = loy_prgrm_number = None
        
        parsed_data.append({
            "First Name": first_name,
            "Last Name": last_name,
            "Sex": sex,
            "Document": document,
            "Flight Date": flight["Date"],
            "Flight Number": flight["Flight"],
            "Share Code": flight["Codeshare"],
            "ArrivalCity": flight["Arrival"]["City"],
            "ArrivalAirport": flight["Arrival"]["Airport"], 
            "DepartureCity": flight["Departure"]["City"],   
            "DepartureAirport": flight["Departure"]["Airport"],
            "NickName": profile.get("NickName", None), 
            "LoyPrgrmStatus": loy_prgrm_status,
            "LoyPrgrmCode": loy_prgrm_code,
            "LoyPrgrmNumber": loy_prgrm_number
        })

# Преобразование списка в pandas DataFrame
df_json = pd.DataFrame(parsed_data)

# Шаг 3: Чтение файла Sirena-export-fixed.tab
colspecs = [
    (0, 60),   # PaxName
    (60, 72),  # PaxBirthDate
    (72, 84),  # DepartDate
    (84, 96),  # DepartTime
    (96, 108),  # ArrivalDate
    (108, 120),  # ArrivalTime
    (120, 126),  # Flight
    (126, 132),  # CodeSh
    (132, 138),  # From
    (138, 144),  # Dest
    (144, 150),  # Code
    (150, 168),  # e-Ticket
    (168, 180), # TravelDoc
    (180, 186),# Seat
    (186, 192),# Meal
    (192, 198),# TrvCls
    (198, 216),# Fare  Baggage
    (216, 240),# PaxAdditionalInfo
    (240, 246),# Unnamed: 18
    (246, 276), # Unnamed: 19
    (276, 336) # AgentInfo
]

df_tab = pd.read_fwf(tab_file_path, colspecs=colspecs, dtype={'Unnamed: 19': 'str', 'Unnamed: 18': 'str'})

# Удаляем 'FF#' в столбце Unnamed: 18, оставляем только программный код (например, 'DT')
df_tab['Unnamed: 18'] = df_tab['Unnamed: 18'].str.replace('FF#', '', regex=False)

# Шаг 4: Разделение PaxName на три части (Last Name, First Name, Second Name)
df_tab[['Last Name', 'First Name', 'Second Name']] = df_tab['PaxName'].str.split(expand=True, n=2)

# Шаг 5: Переименование и объединение столбцов из Sirena с основным датасетом
df_tab = df_tab.rename(columns={
    'PaxBirthDate': 'Birth Date',
    'Flight': 'Flight Number',
    'CodeSh': 'Share Code',
    'From': 'DepartureAirport',
    'Dest': 'ArrivalAirport',
    'Code': 'BookingCode',
    'e-Ticket': 'TicketNumber',
    'TravelDoc': 'Document',
    'Fare  Baggage': 'Baggage',
    'DepartDate': 'Flight Date',
    'DepartTime': 'FlightTime',
    'Unnamed: 19': 'LoyPrgrmNumber',
    'Unnamed: 18': 'LoyPrgrmCode'
})

# Добавление недостающих колонок в df_csv
missing_columns = ['AgentInfo', 'PaxAdditionalInfo', 'Meal', 'Seat', 'ArrivalDate', 'ArrivalTime', 'TrvCls']
for col in missing_columns:
    df_csv[col] = 'N/A'

# Шаг 6: Чтение и обработка XML-файла
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Создаем список для хранения данных
data = []

# Проходим по каждому пользователю
for user in root.findall('user'):
    uid = user.get('uid')  # ID пользователя
    first_name = user.find('name').get('first')  # Имя пользователя
    last_name = user.find('name').get('last')  # Фамилия пользователя
    
    # Проходим по каждой карточке
    for card in user.findall('cards/card'):
        card_number = card.get('number')  # Номер карты
        bonus_program = card.find('bonusprogramm').text  # Бонусная программа
        
        # Проходим по каждой активности (рейсу)
        for activity in card.findall('activities/activity'):
            flight_code = activity.find('Code').text  # Код рейса
            flight_date = activity.find('Date').text  # Дата рейса
            departure = activity.find('Departure').text  # Место вылета
            arrival = activity.find('Arrival').text  # Место прилета
            fare = activity.find('Fare').text  # Тип тарифа
            
            # Добавляем все данные в список
            data.append({
                'User uid': uid,
                'First Name': first_name,
                'Last Name': last_name,
                'CardNumber': card_number,
                'BonusProgram': bonus_program,
                'Flight Number': flight_code,
                'Flight Date': flight_date,
                'DepartureAirport': departure,
                'ArrivalAirport': arrival
            })

# Преобразуем список в DataFrame
df_xml = pd.DataFrame(data)

# Разделяем CardNumber на два столбца (LoyPrgrmCode и LoyPrgrmNumber)
df_xml[['LoyPrgrmCode', 'LoyPrgrmNumber']] = df_xml['CardNumber'].str.split(expand=True)

# Шаг 7: Чтение данных из xlsx_data.csv
df_xlsx = pd.read_csv(xlsx_data_path)

# Шаг 8: Объединение всех DataFrame'ов
df_combined = pd.concat([df_csv, df_json, df_tab, df_xml, df_xlsx], ignore_index=True)

# Удаляем ненужный столбец PaxName
df_combined = df_combined.drop(columns=["PaxName", "CardNumber"], errors='ignore')

# Шаг 9: Сохранение результата в CSV
df_combined.to_csv(output_file_path, index=False)

# Вывод времени работы программы
end_time = time.time()
execution_time = end_time - start_time
print(f"Время работы программы: {execution_time:.2f} секунд")


In [ ]:
# Сохраняем объединённый DataFrame в CSV-файл
df_combined.to_csv(output_file_path, index=False)

# Проверка размера объединённого DataFrame
df_combined_result = df_combined
print(f"Размер объединённого DataFrame (после объединения): {df_combined_result.shape}")

# Сбрасываем индексы и создаём новые
df_combined_result = df_combined_result.reset_index(drop=True)

# Устанавливаем параметр для отображения всех колонок
pd.set_option('display.max_columns', None)

df_combined_result.head(1055981)


#Последняя рабочая версия #Последняя рабочая версия #Последняя рабочая версия
#Последняя рабочая версия #Последняя рабочая версия #Последняя рабочая версия